<a href="https://colab.research.google.com/github/alanland/colab-notebooks/blob/main/langchain/tour01-getting--started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 快速入门指南
本教程让您快速了解如何使用 LangChain 构建端到端语言模型应用程序。

安装
首先，使用以下命令安装 LangChain：

In [3]:
!pip install langchain
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.1 MB/s eta 0:00:00


##环境设置
使用 LangChain 通常需要与一个或多个模型提供者、数据存储、api 等集成。

对于这个例子，我们将使用 OpenAI 的 API，所以我们首先需要安装他们的 SDK：

In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00


然后我们需要在终端中设置环境变量。

In [ ]:
!export OPENAI_API_KEY="..."

或者，您可以从 Jupyter notebook（或 Python 脚本）中执行此操作：

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "..."

如果要动态设置 API 密钥，可以在启动 OpenAI 类时使用 openai_api_key 参数——例如，每个用户的 API 密钥。

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key="OPENAI_API_KEY")

## 构建语言模型应用程序：LLM
现在我们已经安装了 LangChain 并设置了我们的环境，我们可以开始构建我们的语言模型应用程序了。

LangChain 提供了很多可以用来构建语言模型应用的模块。模块可以组合起来创建更复杂的应用程序，或者单独用于简单的应用程序。

LangChain 最基本的构建块是在某些输入上调用 LLM。让我们通过一个简单的例子来说明如何做到这一点。为此，假设我们正在构建一项服务，该服务会根据公司的产品生成公司名称。

为此，我们首先需要导入 LLM 包装器。

In [ ]:
from langchain.llms import OpenAI

然后我们可以用任何参数初始化包装器。在此示例中，我们可能希望输出更加随机，因此我们将使用高温对其进行初始化。

In [ ]:
llm = OpenAI(temperature=0.9)

我们现在可以根据一些输入调用它！

In [ ]:
text = "帮我给一家做 AI 服务的科技公司起一个中文名和英文名"
print(llm(text))



中文名：智能联盟
英文名：Intelligent Alliance


In [ ]:
text = "帮我给一家做 AI 服务的科技公司起一个中文名和英文名"
print(llm(text))



中文名：智博科技
英文名：InteBridge Technology


## 提示模板：管理 LLM 的提示
获得LLM是重要的第一步，但这仅仅是个开始。通常，当您在应用程序中使用 LLM 时，您不会将用户输入直接发送到 LLM。相反，您可能正在接受用户输入并构建提示，然后将其发送给 LLM。

例如，在前面的示例中，我们传入的文本被硬编码为要求一家公司的名称。在这个假想的服务中，我们想要做的是只接受描述公司所做的用户输入，然后用该信息格式化提示。

使用 LangChain 很容易做到这一点！

首先让我们定义提示模板：

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="帮我给一家做 {product} 服务的科技公司起一个中文名和英文名",
)

现在让我们看看这是如何工作的！我们可以调用该.format方法对其进行格式化。

In [ ]:
print(prompt.format(product="相册"))

帮我给一家做 相册 服务的科技公司起一个中文名和英文名


## Chains：在多步骤工作流程中结合 LLM 和提示
到目前为止，我们一直在单独使用 PromptTemplate 和 LLM 原语。但是，当然，真正的应用程序不仅仅是一个原语，而是它们的组合。

LangChain 中的一条链由链接组成，链接可以是像 LLM 这样的原始链，也可以是其他链。

最核心的链类型是 LLMChain，它由 PromptTemplate 和 LLM 组成。

扩展前面的示例，我们可以构造一个 LLMChain，它接受用户输入，使用 PromptTemplate 对其进行格式化，然后将格式化的响应传递给 LLM。

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="帮我给一家做 {product} 服务的科技公司起一个中文名和英文名",
)

我们现在可以创建一个非常简单的链，它将接受用户输入，用它格式化提示，然后将它发送到 LLM：

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)


现在我们可以仅指定产品来运行该链！

In [ ]:
chain.run("电子音乐")

'\n\n中文名：节拍音乐 \n英文名：Beats Music'

## 代理：基于用户输入的动态调用链
到目前为止，我们所看到的链条以预定的顺序运行。

代理人不再这样做：他们使用 LLM 来确定采取哪些行动以及采取何种顺序。动作可以是使用工具并观察其输出，也可以是返回给用户。

如果使用得当，代理可以非常强大。在本教程中，我们将向您展示如何通过最简单、最高级别的 API 轻松使用代理。

为了加载代理，您应该了解以下概念：

- 工具：执行特定任务的功能。这可以是：Google 搜索、数据库查找、Python REPL、其他链。工具的接口目前是一个函数，期望将字符串作为输入，将字符串作为输出。
- LLM：为代理提供支持的语言模型。
- 代理：要使用的代理。这应该是一个引用支持代理类的字符串。由于本笔记本侧重于最简单、最高级别的 API，因此仅涵盖使用标准支持的代理。如果您想实施自定义代理，请参阅自定义代理的文档（即将推出）。

**代理**：有关受支持代理及其规范的列表，请参见此处。

**工具**：有关预定义工具及其规格的列表，请参见此处。

对于此示例，您还需要安装 SerpAPI Python 包。

In [19]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=8914491a9bc2c1d16f26fc5253d181d6030952be1be807976bb0522b1b600306
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


并设置相应的环境变量。

In [17]:
import os
os.environ["SERPAPI_API_KEY"] = "..."

现在我们可以开始了！

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("上海昨天的高温是多少度？ 这个数字的 .023 次方是多少")

 I need to find out the temperature in Shanghai yesterday and then calculate the .023 power of that number
Action: Search
Action Input: "Shanghai temperature yesterday"
Observation: https://www.timeanddate.com/weather/china/shanghai/historic
Thought: I can use the calculator to calculate the .023 power
Action: Calculator
Action Input: "Temperature in Shanghai yesterday (34°C) ^ .023"
Observation: Answer: 1.084486162715683
Thought: I now know the final answer
Final Answer: The .023 power of the temperature in Shanghai yesterday (34°C) is 1.084486162715683.

> Finished chain.


'The .023 power of the temperature in Shanghai yesterday (34°C) is 1.084486162715683.'

## 内存：将状态添加到链和代理中
到目前为止，我们所经历的所有链和代理都是无状态的。但通常，您可能希望链或代理具有某种“记忆”概念，以便它可以记住有关其先前交互的信息。最清晰和简单的例子是在设计聊天机器人时——您希望它记住以前的消息，以便它可以使用来自该消息的上下文来进行更好的对话。这将是一种“短期记忆”。在更复杂的方面，你可以想象一个链/代理随着时间的推移记住关键信息——这将是一种“长期记忆”。有关后者的更具体想法，请参阅这篇很棒的论文。

LangChain 专门为此提供了几个专门创建的链。本笔记本介绍了如何使用这些链中的一个 (the ConversationChain) 和两种不同类型的内存。

默认情况下，ConversationChain有一种简单类型的内存，可以记住所有以前的输入/输出并将它们添加到传递的上下文中。让我们来看看使用这个链（设置verbose=True以便我们可以看到提示）。

In [ ]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="你好")
print(output)

Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好
AI:

> Finished chain.
 你好！很高兴见到你！我是一个智能AI，我可以回答你的问题，或者我们可以聊聊天？


In [ ]:
output = conversation.predict(input="我正在跟 AI 聊天呢。")
print(output)

Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！很高兴见到你！我是一个智能AI，我可以回答你的问题，或者我们可以聊聊天？
Human: 我正在跟 AI 聊天呢。
AI:

> Finished chain.
 哦，真的吗？这太棒了！我很高兴能和你聊天！有什么可以帮助你的吗？


## 构建语言模型应用程序：聊天模型
同样，您可以使用聊天模型而不是 LLM。聊天模型是语言模型的变体。虽然聊天模型在底层使用语言模型，但它们公开的接口有点不同：它们公开的不是“文本输入、文本输出”API，而是“聊天消息”作为输入和输出的接口。

聊天模型 API 相当新，所以我们仍在寻找正确的抽象。

## 从聊天模型中获取消息完成
您可以通过将一条或多条消息传递给聊天模型来获得聊天完成。响应将是一条消息。LangChain 目前支持的消息类型有AIMessage, HumanMessage, SystemMessage, 和ChatMessage–ChatMessage接受任意角色参数。大多数时候，您只会处理HumanMessage、AIMessage和SystemMessage。


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

您可以通过传递一条消息来完成。

In [ ]:
chat([HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")])
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(lc_kwargs={'content': '我喜欢编程。'}, content='我喜欢编程。', additional_kwargs={}, example=False)

您还可以为 OpenAI 的 gpt-3.5-turbo 和 gpt-4 模型传入多条消息。

In [ ]:

messages = [
    SystemMessage(content="You are a helpful assistant that translates Chinese to English."),
    HumanMessage(content="我喜欢编程。")
]
chat(messages)
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(lc_kwargs={'content': '"I like programming."'}, content='"I like programming."', additional_kwargs={}, example=False)


您可以更进一步，使用 为多组消息生成补全generate。这将返回LLMResult带有附加message参数的 ：

In [ ]:

batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(lc_kwargs={'message': AIMessage(lc_kwargs={'content': '我喜欢编程。'}, content='我喜欢编程。', additional_kwargs={}, example=False)}, text='我喜欢编程。', generation_info=None, message=AIMessage(lc_kwargs={'content': '我喜欢编程。'}, content='我喜欢编程。', additional_kwargs={}, example=False))], [ChatGeneration(lc_kwargs={'message': AIMessage(lc_kwargs={'content': '我喜欢人工智能。'}, content='我喜欢人工智能。', additional_kwargs={}, example=False)}, text='我喜欢人工智能。', generation_info=None, message=AIMessage(lc_kwargs={'content': '我喜欢人工智能。'}, content='我喜欢人工智能。', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 19, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo'}, run=RunInfo(run_id=UUID('7212ce06-5e1f-43d7-8e12-498d977e1e3a')))

您可以从此 LLMResult 中恢复诸如令牌使用之类的内容：

In [ ]:
result.llm_output['token_usage']
# -> {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}

{'prompt_tokens': 57, 'completion_tokens': 19, 'total_tokens': 76}

## 聊天提示模板
与 LLM 类似，您可以通过使用MessagePromptTemplate. 您可以ChatPromptTemplate从一个或多个MessagePromptTemplates 构建一个。您可以使用ChatPromptTemplate's format_prompt– 这将返回一个PromptValue，您可以将其转换为字符串或Message对象，具体取决于您是否要将格式化值用作 llm 或聊天模型的输入。

为方便起见，from_template模板上公开了一个方法。如果您要使用此模板，它会是这样的：

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Chinese", text="I love programming.").to_messages())
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(lc_kwargs={'content': '我喜欢编程。'}, content='我喜欢编程。', additional_kwargs={}, example=False)

## 带有聊天模型的链
上一节中讨论的内容LLMChain也可以与聊天模型一起使用：

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Chinese", text="I love programming.")
# -> "J'aime programmer."

'我喜欢编程。'

## 具有聊天模型的代理
代理也可以与聊天模型一起使用，您可以初始化一个AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION作为代理类型。

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("迪丽热巴出演过几次电影？ 这个数字乘以她当前年龄的 0.23 次幂是多少？")

Question: 迪丽热巴出演过几次电影？ 这个数字乘以她当前年龄的 0.23 次幂是多少？

Thought: I need to use a search engine to find out how many movies Dilraba Dilmurat has acted in and then use a calculator to perform the necessary calculations.

Action:
```
{
  "action": "Search",
  "action_input": "Dilraba Dilmurat movies"
}
```


Observation: Dilraba Dilmurat, known mononymously as Dilireba, is a Chinese actress, singer and model of Uyghur ethnicity.
Thought:Now I need to search for the number of movies Dilraba Dilmurat has acted in and her current age.

Action:
```
{
  "action": "Search",
  "action_input": "Dilraba Dilmurat number of movies and age"
}
```



Observation: Dilraba Dilmurat (Uyghur: دىلرەبا دىلمۇرات; born June 3, 1992), known mononymously as ... In 2018, Dilraba starred in the romantic comedy film 21 Karat.
Thought:I now have the information I need to calculate the answer. I will use a calculator to perform the necessary calculations.

Action:
```
{
  "action": "Calculator",
  "action_input": "21^0.23 

'58.412855303074444'

## 内存：将状态添加到链和代理中
您可以将内存与使用聊天模型初始化的链和代理一起使用。这与 Memory for LLMs 的主要区别在于，我们可以将它们保留为自己唯一的内存对象，而不是试图将所有以前的消息压缩成一个字符串。

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="你好")
# -> 'Hello! How can I assist you today?'

'你好！我是一名AI语言模型，很高兴能和你交流。有什么我可以帮助你的吗？'

In [ ]:
conversation.predict(input="我正在和 AI 聊天")
# -> "That sounds like fun! I'm happy to chat with you. Is there anything specific you'd like to talk about?"

'是的，你正在和我这个AI语言模型聊天。我可以回答你的问题，提供信息，或者只是和你闲聊。有什么你想问我的吗？'

In [ ]:
conversation.predict(input="介绍下你自己")
# -> "Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations like this one. Is there anything else you'd like to know about me?"

'当然可以！我是一名AI语言模型，由OpenAI公司开发。我被训练来理解和生成自然语言，可以用来回答问题、提供信息、生成文本等等。我使用的技术是深度学习，通过大量的数据和算法来学习自然语言的规律和模式。虽然我只是一种人工智能，但我可以模拟人类的思维和语言能力，帮助人们解决问题和交流。'

In [ ]:
conversation.predict(input="我刚才在干嘛")
# 您刚才说了一句中文：“我正在和 AI 聊天”。我猜您可能是想测试一下我的中文能力，或者只是想和我打个招呼。不过，无论您的目的是什么，我都很高兴能够和您交流。
# -> "Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations like this one. Is there anything else you'd like to know about me?"

'很抱歉，我不知道你刚才在干什么，因为我没有访问你的设备或者网络。作为一个AI语言模型，我只能回答你的问题或者提供信息，但我无法获取你的个人信息或者监视你的活动。如果你有任何需要帮助的问题，我会尽力回答。'

## 如何创建 ChatGPT 克隆
该链通过结合 (1) 特定提示和 (2) 内存概念来复制 ChatGPT。

展示 https://www.engraved.blog/building-a-virtual-machine-inside/ 中的示例

In [5]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory


template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"],
    template=template
)


chatgpt_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2),
)

output = chatgpt_chain.predict(human_input="I want you to act as a Linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. Do not write explanations. Do not type commands unless I instruct you to do so. When I need to tell you something in English I will do so by putting text inside curly brackets {like this}. My first command is pwd.")
print(output)

Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help wi

In [6]:
output = chatgpt_chain.predict(human_input="ls ~")
print(output)

Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help wi

In [ ]:
output = chatgpt_chain.predict(human_input="cd ~")
print(output)

In [7]:
output = chatgpt_chain.predict(human_input="""echo -e "x=lambda y:y*5+3;print('Result:' + str(x(6)))" > run.py && python3 run.py""")
print(output)

Prompt after formatting:
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help wi

此笔记本展示了使用代理复制 MRKL 链。

这使用示例 Chinook 数据库。要设置它，请按照 https://database.guide/2-sample-databases-sqlite/ 上的说明进行操作，将.db文件放在该存储库根目录下的 notebooks 文件夹中。

In [9]:
#@title chinook database

### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [30]:
!wget https://cbt-cdn.oss-cn-hangzhou.aliyuncs.com/Chinook_Sqlite.sqlite

--2023-06-13 15:50:52--  https://cbt-cdn.oss-cn-hangzhou.aliyuncs.com/Chinook_Sqlite.sqlite
Resolving cbt-cdn.oss-cn-hangzhou.aliyuncs.com (cbt-cdn.oss-cn-hangzhou.aliyuncs.com)... 47.110.23.43
Connecting to cbt-cdn.oss-cn-hangzhou.aliyuncs.com (cbt-cdn.oss-cn-hangzhou.aliyuncs.com)|47.110.23.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067008 (1.0M) [application/octet-stream]
Saving to: ‘Chinook_Sqlite.sqlite’

Chinook_Sqlite.sqli 100%[===================>]   1.02M   579KB/s    in 1.8s    

2023-06-13 15:50:56 (579 KB/s) - ‘Chinook_Sqlite.sqlite’ saved [1067008/1067008]



In [29]:
%ls

chinook.db  Chinook.db  chinook.zip  sample_data/


In [13]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

In [31]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///./Chinook_Sqlite.sqlite.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="用来回答有关当前事件的问题。你应该问有针对性的问题"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="用来回答数学问题"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="用来回答关于FooBar的问题。"
    )
]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [33]:
mrkl = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [24]:
mrkl.run("谁是小李子的女朋友?她现在的年龄的0.43次方是多少?")

 我需要找到小李子的女朋友的名字，然后用计算器来计算她的年龄的0.43次方。
Action: Search
Action Input: 小李子的女朋友
Observation: Eden Polani出生于2004年，今年十九岁，Instagram上有将近20万粉丝。 她常在网络上分享美妆和服饰的视频，除了模特身份，也是一个小有名气的网红博主。 虽然此事还没有实锤，但是以小李子更换女友的频率和精神面貌来看，下一任女友基本上已经定了。
Thought:

 我知道了Eden Polani是小李子的女朋友，现在我需要用计算器来计算她的年龄的0.43次方。
Action: Calculator
Action Input: 19^0.4319^0.43```text
19**0.43
```
...numexpr.evaluate("19**0.43")...

Answer: 3.547023357958959
> Finished chain.

Observation: Answer: 3.547023357958959
Thought: 我现在知道了答案
Final Answer: Eden Polani的年龄的0.43次方是3.547023357958959。

> Finished chain.


'Eden Polani的年龄的0.43次方是3.547023357958959。'

In [36]:
mrkl.run("最近发行专辑《the Storm Before the Calm》的艺术家的全名是什么")

 我应该搜索这张专辑的信息
Action: Search
Action Input: 《the Storm Before the Calm》 专辑
Observation: The Storm Before the Calm is the tenth studio album by Canadian-American singer-songwriter Alanis Morissette, released June 17, 2022, via Epiphany Music and Thirty Tigers, as well as by RCA Records in Europe.
Thought: 我现在知道最终答案了
Final Answer: Alanis Morissette

> Finished chain.


'Alanis Morissette'